<a href="https://colab.research.google.com/github/kelvincai522/houdini/blob/master/uniio_houdini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#writer density
node = hou.pwd()
geo = node.geometry()

vol = geo.prims()[0]
# Add code to modify contents of geo.
# Use drop down menu to select examples.

import collections
import uniio
import numpy as np

header = collections.OrderedDict()
header['dimX'] = vol.resolution()[0]
header['dimY'] = vol.resolution()[1]
header['dimZ'] = vol.resolution()[2]
header['gridType'] = 1
header['elementType'] = 1
header['bytesPerElement'] = 4
header['info'] = ''
header['dimT'] = 0
header['timestamp'] = 0

voxels = vol.allVoxels()

data = np.asarray(voxels)
channels = 1
dimensions = [header['dimZ'], header['dimY'], header['dimX'], channels]

data = data.reshape( *dimensions, order='C')

uniio.writeUni("/tmp/test.uni", header, data)

print header
print np.shape(data)

#####################################################################
#writer velocity
node = hou.pwd()
geo = node.geometry()

volx = geo.prims()[0]
voly = geo.prims()[1]
volz = geo.prims()[2]
# Add code to modify contents of geo.
# Use drop down menu to select examples.

import collections
import uniio
import numpy as np

header = collections.OrderedDict()
header['dimX'] = volx.resolution()[0]
header['dimY'] = volx.resolution()[1]
header['dimZ'] = volx.resolution()[2]
header['gridType'] = 12
header['elementType'] = 2
header['bytesPerElement'] = 12
header['info'] = ''
header['dimT'] = 0
header['timestamp'] = 0

xvoxels = np.asarray(volx.allVoxels())
xvoxels = np.reshape(xvoxels, (1, len(xvoxels)))
#print xvoxels
yvoxels = np.asarray(voly.allVoxels())
yvoxels = np.reshape(yvoxels, (1, len(yvoxels)))
#print yvoxels
zvoxels = np.asarray(volz.allVoxels())
zvoxels = np.reshape(zvoxels, (1, len(zvoxels)))
#print zvoxels

data = np.concatenate((xvoxels.T, yvoxels.T, zvoxels.T),axis=1)
#data = np.hstack((xvoxels, yvoxels, zvoxels))
#print data
channels = 3
dimensions = [header['dimZ'], header['dimY'], header['dimX'], channels]

data = data.reshape( *dimensions, order='C')
print data

uniio.writeUni("/tmp/test.uni", header, data)

#print header
print np.shape(data)



#####################################################################
#reader
node = hou.pwd()
geo = node.geometry()

# Add code to modify contents of geo.
# Use drop down menu to select examples.
#vol = geo.prim(0)

import uniio
import numpy as np
filepath = node.parm("unifile").eval()
header, content = uniio.readUni(filepath)

#print content
#print header

dimX = header["dimX"]
dimY = header["dimY"]
dimZ = header["dimZ"]

if (header['elementType'] != 2):
    vol = geo.createVolume(dimX, dimY, dimZ)
    content = np.reshape(content, (dimX*dimY*dimZ))
    vol.setAllVoxelsFromString(content)
else:
#    print content
    x, y, z = content[:,:,:,0], content[:,:,:,1], content[:,:,:,2]
#    print x
    x = np.reshape(x, (dimX*dimY*dimZ))
    x = x.astype(np.float32)
    y = np.reshape(y, (dimX*dimY*dimZ))
    y = y.astype(np.float32)
    z = np.reshape(z, (dimX*dimY*dimZ))
    z = z.astype(np.float32)
    vol = geo.createVolume(dimX, dimY, dimZ)
    vol.setAllVoxelsFromString(x)
    vol = geo.createVolume(dimX, dimY, dimZ)
    vol.setAllVoxelsFromString(y)
    vol = geo.createVolume(dimX, dimY, dimZ)
    vol.setAllVoxelsFromString(z)
#print header


